In [11]:
import pandas as pd

In [1]:
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd

# SQLiteデータベースファイルへのパス
db_path = 'sqlite:///pop-make-up_DB.db'

# SQLAlchemyエンジンの作成
engine = create_engine(db_path)

# メタデータの作成
metadata = MetaData()

def select_tbl(mytable):
  # テーブルの定義
  your_table = Table(mytable, metadata, autoload_with=engine)

  # SELECTクエリの作成
  select_query = your_table.select()

  # クエリの実行
  with engine.connect() as connection:
      result = connection.execute(select_query)
      df = pd.DataFrame(result.fetchall(), columns=result.keys())

  return df


ModuleNotFoundError: No module named 'sqlalchemy'

In [19]:
stores = select_tbl('stores')
trn = select_tbl('transaction_records')
trn

,ID,USER_ID,PRD_ID,MY_COUPON_ID,DATE
0,1,6,18,3,2024-01-01
1,2,6,4,3,2024-01-01
2,3,6,13,3,2024-01-01
3,4,12,17,1,2024-01-01
4,5,12,16,,2024-01-01
...,...,...,...,...,...
4839,4840,276,13,3,2024-04-24
4840,4841,362,6,1,2024-04-24
4841,4842,369,15,1,2024-04-24
4842,4843,406,3,2,2024-04-24


In [23]:
target_month = 4  # 例として4月を指定
filtered_trn = trn[pd.to_datetime(trn['DATE']).dt.month == target_month]

trn_num_ct = filtered_trn.shape[0]
trn_user_ct = len(filtered_trn['USER_ID'].unique())

print(trn_num_ct)
print(trn_user_ct)

1017
282


In [25]:
filtered_trn['MY_COUPON_ID']


3827    3
3828    3
3829    3
3830     
3831     
       ..
4839    3
4840    1
4841    1
4842    2
4843    1
Name: MY_COUPON_ID, Length: 1017, dtype: object

In [15]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

from crud import select_tbl




stores = select_tbl('stores')
trn = select_tbl('transaction_records')

target_dt = pd.to_datetime('2024-04-01')

this_month_trn = trn[ (trn['DATE'].dt.year == target_dt.year) & (trn['DATE'].dt.month == target_dt.month) ]  #指定月のtransitionデータ

trn_num_ct = this_month_trn.shape[0] #総利用食数

trn_user_ct = len(this_month_trn['USER_ID'].unique()) #利用人数

#ユーザーごとの利用日数
_user_trn_ct = this_month_trn[['USER_ID','DATE']].drop_duplicates()
_user_trn_ct['use_dt_ct'] = 1
user_trn_ct = _user_trn_ct.groupby('USER_ID',as_index=False)['use_dt_ct'].sum() 
user_trn_ct['use_dt_ct_per_week'] = round(user_trn_ct['use_dt_ct']/4,0)
def labeling(x):
  if x>=5:
    return '５回以上'
  else:
    return f'{int(x)}回'
user_trn_ct['label'] = user_trn_ct['use_dt_ct_per_week'].apply(lambda x: labeling(x))
user_trn_ct_sum = user_trn_ct.groupby('label',as_index=False).size()
mean_ct = user_trn_ct['use_dt_ct_per_week'].mean()
user_3over_week = user_trn_ct[user_trn_ct['use_dt_ct_per_week']>=3]
user_ct_3over_week = len(user_3over_week['USER_ID'])
rate_3over_week = user_ct_3over_week/trn_user_ct

target_pre_month = target_dt - pd.DateOffset(months=1)
last_month_trn =  trn[ (trn['DATE'].dt.year == target_pre_month.year) & (trn['DATE'].dt.month == target_pre_month.month) ]
_user_trn_ct_pre = this_month_trn[['USER_ID','DATE']].drop_duplicates()
_user_trn_ct_pre['use_dt_ct'] = 1
user_trn_ct_pre = _user_trn_ct_pre.groupby('USER_ID',as_index=False)['use_dt_ct'].sum() 
user_trn_ct_pre['use_dt_ct_per_week'] = round(user_trn_ct_pre['use_dt_ct']/4,0)
mean_ct_pre = user_trn_ct_pre['use_dt_ct_per_week'].mean()

dif_mean_ct = mean_ct - mean_ct_pre
plus_minus_mean_ct = "+" if mean_ct >= 0 else "-"

In [16]:
user_trn_ct_sum

,label,size
0,0回,239
1,1回,25
2,2回,12
3,3回,6


In [17]:
go.Figure(data=[go.Pie(labels=user_trn_ct_sum['label'], values=user_trn_ct_sum['size'], hole=0.5)])
